In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imshow
from skimage.measure import label
from scipy.ndimage.morphology import distance_transform_edt
from PIL import Image
from numpy import asarray
import os
from skimage.exposure import rescale_intensity
import skimage
from skimage.transform import resize
from csbdeep.utils import Path, normalize
from tifffile import imread, imsave
import sys
#sys.path.insert(0,'../../../../SyMBac')
from SyMBac.misc import resize_mask
from skimage.measure import label
from tqdm.notebook import tqdm
def unet_weight_map(y, wc=None, w0 = 10, sigma = 5):

    """
    Generate weight maps as specified in the U-Net paper
    for boolean mask.

    "U-Net: Convolutional Networks for Biomedical Image Segmentation"
    https://arxiv.org/pdf/1505.04597.pdf

    Parameters
    ----------
    mask: Numpy array
        2D array of shape (image_height, image_width) representing binary mask
        of objects.
    wc: dict
        Dictionary of weight classes.
    w0: int
        Border weight parameter.
    sigma: int
        Border width parameter.

    Returns
    -------
    Numpy array
        Training weights. A 2D array of shape (image_height, image_width).
    """

    labels = label(y)
    no_labels = labels == 0
    label_ids = sorted(np.unique(labels))[1:]

    if len(label_ids) > 1:
        distances = np.zeros((y.shape[0], y.shape[1], len(label_ids)))

        for i, label_id in enumerate(label_ids):
            distances[:,:,i] = distance_transform_edt(labels != label_id)

        distances = np.sort(distances, axis=2)
        d1 = distances[:,:,0]
        d2 = distances[:,:,1]
        w = w0 * np.exp(-1/2*((d1 + d2) / sigma)**2) * no_labels
    else:
        w = np.zeros_like(y)
    if wc:
        class_weights = np.zeros_like(y)
        for k, v in wc.items():
            class_weights[y == k] = v
        w = w + class_weights
    return w

base_directory = "/home/georgeos/Storage/Dropbox (Cambridge University)/Phase/DeLTA_new_training_data_hist_fourier_mixed/"

training_img_dir = base_directory + "convolutions/"
training_img_files = os.listdir(training_img_dir)

masks_dir = base_directory + "masks/"
masks_files = os.listdir(masks_dir)
weightmap_dir = base_directory + "WEIGHTMAPS"

try: 
    os.mkdir(base_directory + "preprocessed")
    os.mkdir(base_directory + "preprocessed/CROPPED_FILTERED")
    os.mkdir(base_directory + "preprocessed/CROPPED_MASKS")
    os.mkdir(base_directory + "preprocessed/WEIGHTMAPS")
except:
    pass
    
def crop_and_resize(image_dir, resize_shape):
    image = imread(image_dir)
    image = resize(image,resize_shape, order=0, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=False, anti_aliasing_sigma=None)
    return image
    
  



FileNotFoundError: [Errno 2] No such file or directory: '/home/georgeos/Storage/Dropbox (Cambridge University)/Phase/DeLTA_new_training_data_hist_fourier_mixed/convolutions/'

In [4]:
from skimage.measure import label
#from skimage.segmentation import find_boundaries
#from skimage.transform import resize
#def resize_mask(mask, resize_shape, ret_label):
#    """
#    Resize masks while maintaining their connectivity and values
#    """
#    #labeled_mask = label(mask>0,connectivity=1)
#    labeled_mask = mask
#    labeled_mask = resize(labeled_mask,resize_shape, order=0, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=False, anti_aliasing_sigma=None).astype(int)
#    mask = resize(mask,resize_shape, order=0, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=False, anti_aliasing_sigma=None).astype(int)
#    mask_borders = find_boundaries(labeled_mask,mode="outer", connectivity=2)
#    labeled_mask = np.where(mask_borders, 0,labeled_mask)
#    assert len(np.unique(mask)) == len(np.unique(label(labeled_mask > 0))), "you joined some masks"
#    if ret_label:
#        return labeled_mask
#    else:
#        return labeled_mask > 0

In [5]:
resize_shape = (256,48)

for x in tqdm(range(len(training_img_files))):
    cropped = crop_and_resize(training_img_dir + training_img_files[x], resize_shape)
    cropped = normalize(cropped, 1,99.8, axis=(0,1))
    cropped = Image.fromarray(cropped)
    cropped.save(base_directory + "preprocessed/CROPPED_FILTERED/"+training_img_files[x])
    
for x in tqdm(range(len(masks_files))):
    cropped = crop_and_resize(masks_dir + masks_files[x], resize_shape)
    cropped = np.array(cropped).astype(bool)
    assert len(np.unique(label(imread(masks_dir + masks_files[x]).astype(bool)))) == len(np.unique(label(cropped)))
    imsave(base_directory + "preprocessed/CROPPED_MASKS/"+masks_files[x],cropped)
    
#for x in range(len(masks_files)):
#    cropped = resize_mask(imread((masks_dir + masks_files[x])), resize_shape, ret_label=False)
#    cropped = np.array(cropped).astype(bool)
#    imsave(base_directory + "preprocessed/CROPPED_MASKS/"+masks_files[x],cropped)
    
    
wc = {
    0: 1, # background
    1: 5  # objects
}

for x in tqdm(range(len(masks_files))):
    image = imread(base_directory + "preprocessed/CROPPED_MASKS/" + masks_files[x])
    w = unet_weight_map(image, wc)
    image = w.astype(np.uint8)
    imsave(base_directory + "preprocessed/WEIGHTMAPS/"+masks_files[x],image)

  0%|          | 0/5618 [00:00<?, ?it/s]

  0%|          | 0/5618 [00:00<?, ?it/s]

  0%|          | 0/5618 [00:00<?, ?it/s]